<a href="https://colab.research.google.com/github/Sant-78/Bank-Statement-App/blob/main/Bank_Statement_Formatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Bank Statement TXT → Clean Excel Automation

In [ ]:
# 🚀 Bank Statement TXT → Clean Excel Automation
# Author: [Santosh.Pal]

!pip install pandas openpyxl

In [ ]:


import pandas as pd
import re
from google.colab import files

# --------- CONFIG ---------
UNWANTED_PATTERNS = [
    r"STATEMENT SUMMARY",
    r"VALUE POST DETAILS",
    r"BROUGHT FORWARD",
    r"CARRIED FORWARD",
    r"COUNT\s+\d+",
    r"PAGE NO",
    r"TOLL FREE",
    r"IN CASE YOUR ACCOUNT",
    r"LETTER OF AUTHORITY",
    r"POWER OF ATTORNEY",
    r"^_+$",
    r"^_+\s*_+$",
    r"^VALUE\s+POST\s+DETAILS\s+CHQ\.NO\.\s+DEBIT\s+CREDIT\s+BALANCE\s+DATE\s+DATE$"
]
KEYWORDS = [
    "BY TRF.", "TO TRF.", "NEFT", "RTGS", "ACH", "TRF FROM", "TRF TO", "LOAN CLOSURE"
]

# --------- HELPERS ---------
def indian_format(num):
    if num in ("", None):
        return ""
    try:
        x = float(num)
    except:
        return ""
    s = f"{abs(x):.2f}"
    whole, dec = s.split(".")
    last3 = whole[-3:]
    rest = whole[:-3]
    if rest:
        parts = []
        while len(rest) > 2:
            parts.append(rest[-2:])
            rest = rest[:-2]
        if rest:
            parts.append(rest)
        parts = parts[::-1]
        whole_fmt = ",".join(parts) + "," + last3
    else:
        whole_fmt = last3
    return ("-" if x < 0 else "") + whole_fmt + "." + dec

def format_details(text):
    text = re.sub(
        r"VALUE\s+POST\s+DETAILS\s+CHQ\.NO\.\s+DEBIT\s+CREDIT\s+BALANCE\s+DATE\s+DATE",
        "", text, flags=re.IGNORECASE
    )
    text = re.sub(r"\s+", " ", text.strip())
    for kw in KEYWORDS:
        text = re.sub(kw, kw.upper(), text, flags=re.IGNORECASE)
    return text

def is_unwanted(line):
    for pat in UNWANTED_PATTERNS:
        if re.search(pat, line, flags=re.IGNORECASE):
            return True
    return False

def finalize_transaction(row):
    details = re.sub(r"\s+", " ", (row.get("DetailsRaw") or "").strip())

    bal_matches = list(re.finditer(r"([0-9]{1,3}(?:,[0-9]{2,3})*\.\d{2})\s*Cr\b", details, flags=re.IGNORECASE))
    balance_val = ""
    if bal_matches:
        m = bal_matches[-1]
        balance_val = m.group(1)
        details = (details[:m.start()] + details[m.end():]).strip()

    amt_matches = list(re.finditer(r"([0-9]{1,3}(?:,[0-9]{2,3})*\.\d{2})", details))
    debit_val, credit_val = "", ""
    if amt_matches:
        value_amount = amt_matches[-1].group(1)
        details = details[:amt_matches[-1].start()] + details[amt_matches[-1].end():]
        if details.upper().startswith("BY "):
            credit_val = value_amount
        elif details.upper().startswith("TO "):
            debit_val = value_amount
        else:
            debit_val = value_amount

    details = format_details(details)

    row["Details"] = details
    row["Debit"] = debit_val
    row["Credit"] = credit_val
    row["Balance"] = balance_val
    return row

# --------- MAIN ---------
def convert_txt_to_excel(txt_path, output_path="Formatted_Bank_Statement.xlsx"):
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    tx_rows = []
    current = None
    tx_start = re.compile(r"^(\d{2}/\d{2}/\d{2})\s+(\d{2}/\d{2}/\d{2})\s+(.*)$")

    for raw in lines:
        line = raw.strip()
        if not line or is_unwanted(line):
            continue

        m = tx_start.match(line)
        if m:
            if current:
                current = finalize_transaction(current)
                tx_rows.append(current)
            value_date, post_date, rest = m.groups()
            current = {
                "Value": value_date,
                "Post": post_date,
                "DetailsRaw": rest
            }
        else:
            if current:
                current["DetailsRaw"] = (current.get("DetailsRaw", "") + " " + line).strip()

    if current:
        current = finalize_transaction(current)
        tx_rows.append(current)

    df = pd.DataFrame(tx_rows, columns=["Value", "Post", "Details", "Debit", "Credit", "Balance"])
    df.insert(3, "Chq.No", "")

    for col in ["Value", "Post"]:
        df[col] = pd.to_datetime(df[col], format="%d/%m/%y", errors="coerce").dt.strftime("%d-%m-%Y")

    for col in ["Debit", "Credit", "Balance"]:
        df[col] = df[col].astype(str).str.replace(",", "", regex=False)
        df[col] = df[col].apply(lambda x: indian_format(x))

    df.to_excel(output_path, index=False)
    return output_path

# --------- RUN ---------
uploaded = files.upload()
txt_path = list(uploaded.keys())[0]
out = convert_txt_to_excel(txt_path)
files.download(out)

Saving ONLINESTMT_5495346710_0000000121676364.txt to ONLINESTMT_5495346710_0000000121676364.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# PSB Bank Statement Clean App



In [2]:
import pandas as pd
import re
from google.colab import files

# Step 1: Upload txt file
print("Please upload your bank statement txt file:")
uploaded = files.upload()

# Get the uploaded filename
filename = list(uploaded.keys())[0]

# Step 2: Parse the bank statement correctly
def parse_bank_statement_fixed(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    transactions = []

    for line in lines:
        original_line = line
        line = line.strip()

        # Skip empty lines
        if not line:
            continue

        # Skip header and footer lines
        skip_patterns = [
            'GL.     Value', 'Date    Date', 'Instrmnt', 'Number',
            'Transaction', 'Debit Amount', 'Credit Amount', 'Balance',
            'Entry', 'Verified', 'User Id',
            '----', 'Page Total', 'Page  ', 'Report', 'Service OutLet',
            'Account No', 'Opening Balance', 'Closing Balance',
            'B/F Balance', 'Signature', 'End of Report',
            'Order by GL. Date', 'Limits', 'Draw Power',
            'Peg Review', 'Int Rate', 'Total Credit', 'Total Debit',
            'Gl Sub Head Code', 'Set Id', 'SolId'
        ]

        if any(skip in line for skip in skip_patterns):
            continue

        # Match transaction line: starts with date pattern
        if re.match(r'\d{2}-\d{2}-\d{4}\s+\d{2}-\d{2}-\d{4}', line):
            try:
                # Split the line into parts
                parts = original_line.split()

                if len(parts) < 4:
                    continue

                gl_date = parts[0]
                value_date = parts[1]

                # Find balance (ends with Cr or Dr)
                balance = ''
                balance_idx = -1

                for i in range(len(parts) - 1, -1, -1):
                    if 'Cr' in parts[i] or 'Dr' in parts[i]:
                        balance = parts[i].replace('Cr', '').replace('Dr', '').strip()
                        balance_idx = i
                        break

                if balance_idx == -1 or balance_idx < 3:
                    continue

                # Now identify amounts before balance
                # The original file has columns: GL Date | Value Date | Particulars | Debit | Credit | Balance

                # Get all parts between dates and balance
                middle_parts = parts[2:balance_idx]

                # Separate amounts from particulars
                amounts = []
                particulars_parts = []

                for part in middle_parts:
                    # Check if it's a number (amount)
                    if re.match(r'^[\d,]+\.\d{2}$', part):
                        amounts.append(part)
                    else:
                        particulars_parts.append(part)

                # Join particulars
                particulars = ' '.join(particulars_parts)

                # Assign amounts to debit/credit
                # Logic: Check if this is a debit or credit transaction
                debit = ''
                credit = ''

                if len(amounts) == 1:
                    # Single amount - need to determine if debit or credit
                    # Check the particulars for clues
                    if any(keyword in particulars.upper() for keyword in ['PAYOFF', 'FC INCOME', 'RTGS/', 'NEFT/PSIBN']):
                        # These are typically debit entries
                        debit = amounts[0]
                    else:
                        # Default to credit (like NEFT/YESIG entries)
                        credit = amounts[0]

                elif len(amounts) == 2:
                    # Two amounts = debit and credit
                    # First amount is debit, second is credit
                    debit = amounts[0]
                    credit = amounts[1]

                # Add transaction
                transactions.append({
                    'GL Date': gl_date,
                    'Value Date': value_date,
                    'Particulars': particulars,
                    'Debit Amount': debit,
                    'Credit Amount': credit,
                    'Balance': balance
                })

            except Exception as e:
                continue

    return transactions

# Step 3: Parse the file
print("\nParsing the bank statement...")
transactions = parse_bank_statement_fixed(filename)

# Step 4: Create DataFrame
df = pd.DataFrame(transactions)

# Step 5: Display preview
print(f"\nTotal transactions found: {len(df)}")
print("\nFirst 20 transactions:")
print(df.head(20).to_string(index=False))

# Step 6: Save to CSV
output_filename = filename.replace('.txt', '_converted.csv')
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ CSV file created: {output_filename}")

# Step 7: Download
print("\nDownloading...")
files.download(output_filename)

print("\n✅ Done!")

Please upload your bank statement txt file:


Saving 15371100000058 17 dec.txt to 15371100000058 17 dec (1).txt

Parsing the bank statement...

Total transactions found: 1961

First 20 transactions:
   GL Date Value Date                                        Particulars Debit Amount Credit Amount        Balance
01-12-2025 01-12-2025              NEFT/PSIBN25335057732/CAPRI GLOBAL CA       358.00               1,85,98,680.50
01-12-2025 01-12-2025              NEFT/PSIBN25335061508/CAPRI GLOBAL CA       579.56               1,85,98,100.94
01-12-2025 01-12-2025                 15371100000058 - Payoff Source a/c 53,40,304.00               1,32,57,796.94
01-12-2025 01-12-2025                 15371100000058 - Payoff Source a/c 15,32,512.82               1,17,25,284.12
01-12-2025 01-12-2025                        CG FC INCOME 16121200000999    55,657.07               1,16,69,627.05
01-12-2025 01-12-2025 NEFT/YESIG53350181709/CAPRI GLOBAL CAPITAL LIMITED               31,48,103.00 1,48,17,730.05
02-12-2025 02-12-2025 NEFT/YESIG5336004967

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Done!
